In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import pickle
from tqdm import tqdm
import re
import json
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from collections import Counter

In [2]:
from model import *
from data import *
from utils import *

In [3]:
import os
os. environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device('cuda:0')

## Train

- **set params**

In [4]:
lr = 0.0001
batch_size=128

- **load train data**

In [5]:
news_vert = np.load('../../data2/news_vert.npy')
news_title = np.load('../../data2/news_title.npy')
news_body = np.load('../../data2/news_body.npy')

In [6]:
with open('../../data2/TrainUsers.pkl', 'rb') as f:
    TrainUsers = pickle.load(f)
with open('../../data2/TrainSamples.pkl', 'rb') as f:
    TrainSamples = pickle.load(f)

- **load model**

In [7]:
with open('../../data2/dict.pkl', 'rb') as f:
    _,category_dict,word_dict = pickle.load(f)
with open('../../data2/news.pkl', 'rb') as f:
    news = pickle.load(f)
embedding_matrix = np.load('../../data2/embedding_matrix.npy')

In [8]:
torch.cuda.empty_cache()

In [9]:
model = NRMS(embedding_matrix)
model = model.to(device)

- **begin training**

In [10]:
def acc(y_true, y_hat):
    y_hat = torch.argmax(y_hat, dim=-1)
    tot = y_true.shape[0]
    hit = torch.sum(y_true == y_hat)
    return hit.data.float() * 1.0 / tot

In [11]:
optimizer = optim.Adam(model.parameters(), lr=0.0001) #lr = 0.0001

In [12]:
min_train_loss = 100.0
for ep in range(1,4):
    loss = 0.0
    accuary = 0.0
    cnt = 1
    dset = TrainDataset(TrainUsers, TrainSamples, news_title, news_vert, news_body)
    data_loader = DataLoader(dset, batch_size=128, collate_fn=collate_fn, shuffle=True)
    tqdm_util = tqdm(data_loader)
    for user_feature, news_feature, label in tqdm_util: 
        user_feature = [i.to(device) for i in user_feature]
        news_feature = [i.to(device) for i in news_feature]
        label = label.to(device)
        bz_loss, y_hat = model(user_feature, news_feature, label)
        loss += bz_loss.data.float()
        accuary += acc(label, y_hat)

        optimizer.zero_grad()
        bz_loss.backward()
        optimizer.step()

        if cnt % 10 == 0:
            tqdm_util.set_description('ed: {}, train_loss: {:.5f}, acc: {:.5f}'.format(cnt * batch_size, loss.data / cnt, accuary / cnt))
        cnt += 1
    loss /= cnt
    print(ep, loss)
    torch.save(model.state_dict(), '../../runs/userencoder/NAML-{}.pkl'.format(ep))


ed: 762880, train_loss: 0.58991, acc: 0.67979: 100%|██████████| 5963/5963 [24:23<00:00,  4.07it/s]


1 tensor(0.5898, device='cuda:0')


ed: 762880, train_loss: 0.55482, acc: 0.71245: 100%|██████████| 5963/5963 [24:57<00:00,  3.98it/s]


2 tensor(0.5547, device='cuda:0')


ed: 762880, train_loss: 0.54282, acc: 0.72187: 100%|██████████| 5963/5963 [24:39<00:00,  4.03it/s]


3 tensor(0.5427, device='cuda:0')


## Test

- **load test data**

In [13]:
with open('../../data2/ValidUsers.pkl', 'rb') as f:
    ValidUsers = pickle.load(f)
with open('../../data2/ValidSamples.pkl', 'rb') as f:
    ValidSamples = pickle.load(f)

In [14]:
for ep in range(1,4):
    model = NRMS(embedding_matrix)
    model = model.to(device)
    model.load_state_dict(torch.load('../../runs/userencoder/NAML-{}.pkl'.format(ep)))
    model.eval() 
    # save new embedding matrix
    np.save('../../data2/embedding_matrix{}.npy'.format(ep), model.embed.weight.data.cpu().numpy())
    

    n_dset = news_dataset(news_title, news_vert, news_body)
    news_data_loader = DataLoader(n_dset, batch_size=512, collate_fn=news_collate_fn, shuffle=False)

      
    news_scoring = []
    torch.cuda.empty_cache()
    with torch.no_grad():
        for news_feature in tqdm(news_data_loader): 
            news_feature = [i.to(device) for i in news_feature]
            news_vec = model.news_encoder(news_feature)
            news_vec = news_vec.to(torch.device("cpu")).detach().numpy()
            news_scoring.extend(news_vec)
    news_scoring = np.array(news_scoring)
    np.save('../../data2/news_scoring{}.npy'.format(ep), news_scoring)
    
    u_dset = UserDataset(news_scoring, ValidUsers)
    user_data_loader = DataLoader(u_dset, batch_size=128, shuffle=False)

    user_scoring = []
    with torch.no_grad():
        for user_feature in tqdm(user_data_loader): 
            user_feature = user_feature.to(device)
            user_vec = model.user_encoder(user_feature)
            user_vec = user_vec.to(torch.device("cpu")).detach().numpy()
            user_scoring.extend(user_vec)
    user_scoring = np.array(user_scoring)
    np.save('../../data2/global_user_embed{}.npy'.format(ep),np.mean(user_scoring,axis=0))
    g = evaluate(user_scoring,news_scoring, ValidSamples)
    print(ep)
    print('AUC\t', 'MRR\t', 'nDCG5\t', 'nDCG10\t','CTR1\t','CTR10\t')
    print(g)

100%|██████████| 100000/100000 [01:29<00:00, 1117.10it/s]


0 0.0
1
AUC	 MRR	 nDCG5	 nDCG10	 CTR1	 CTR10	
(0.6407997589410694, 0.23081682854624697, 0.2535878624744038, 0.33227711742274696, 0.1454, 0.11970133333333333)


100%|██████████| 100000/100000 [01:30<00:00, 1098.98it/s]


0 0.0
2
AUC	 MRR	 nDCG5	 nDCG10	 CTR1	 CTR10	
(0.6460787629807131, 0.23452229193374297, 0.2586624248644371, 0.3377814121898208, 0.14922, 0.12179533333333331)


100%|██████████| 100000/100000 [01:29<00:00, 1118.09it/s]

0 0.0
3
AUC	 MRR	 nDCG5	 nDCG10	 CTR1	 CTR10	
(0.6464411767491163, 0.23536873814293183, 0.25995044819703567, 0.33848005066956344, 0.14956, 0.12227333333333334)
